<a href="https://colab.research.google.com/github/gpandu/CodeGenGPT/blob/main/CodeGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training GPT model is done using following steps.
1. Load and prepare dataset for tokenizer training.
2. Train BPE tokenizer from scratch.
3. Prepare tensorflow dataset with generator.
4. Create layer and model classes for GPT



*   We will use tokenizers library from HuggingFace to train tokenizer from scratch.
*   We will also use datasets to load the python "code_search_net" dataset. It has ~410k of training records. If we load the load at once we will run out of RAM, so we will take advantage streaming the batches.




In [1]:
!pip install tokenizers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00


In [2]:
import math
import numpy as np
import tensorflow as tf
from datasets import load_dataset

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


*   Load the "code_search_net" adn we can check details of the dataset like below.



In [3]:
# load code dataset
raw_dataset = load_dataset("code_search_net", "python")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})



*   Create a generator to load the data in batches for training tokenizer.

   >  Loading all the data at once may cause out of memory error.







In [5]:
tokenizer_batch_len = 1000
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), tokenizer_batch_len):
        samples = dataset[start_idx : start_idx + tokenizer_batch_len]
        yield samples["whole_func_string"]


In [6]:
#check if we are able to iterate over the dataset.
iterat = iter(get_training_corpus())
next(iterat)[10]

'def get_mean_and_stddevs(self, sites, rup, dists, imt, stddev_types):\n        """\n        See :meth:`superclass method\n        <.base.GroundShakingIntensityModel.get_mean_and_stddevs>`\n        for spec of input and result values.\n        """\n        # extracting dictionary of coefficients specific to required\n        # intensity measure type.\n        C = self.COEFFS[imt]\n        if isinstance(imt, PGA):\n            imt_per = 0.0\n        else:\n            imt_per = imt.period\n        # Fix site parameters for consistent dS2S application.\n        sites.vs30 = np.array([250])\n        sites.z1pt0 = np.array([330])\n        # intensity on a reference soil is used for both mean\n        # and stddev calculations.\n        ln_y_ref = self._get_ln_y_ref(rup, dists, C)\n        # exp1 and exp2 are parts of eq. 7\n        exp1 = np.exp(C[\'phi3\'] * (sites.vs30.clip(-np.inf, 1130) - 360))\n        exp2 = np.exp(C[\'phi3\'] * (1130 - 360))\n        # v1 is the period dependent sit

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "more" , "over"

> "more" and "over" are likely to be more frequent than moreover


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   

* We will use BPE(Byte Pair Encoding) to train tokenizer on "code_search_net" python Dataset.
*   For more information on BPE can be found here. https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt


In [7]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# "<|endoftext|>" will used to stop the sequence generation during inference. This is also
#  a way telling GPT to learn to about the end of the sequence
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["PAD","<|endoftext|>"])
#Train the tokenizer using BPE trainer, loads the data in batches
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)


*    Check if we are able to tokenize and encode the data using trained BPE tokenizer
*   Actual maximum sequence length MAX_SEQ_LENGTH = 256, we will add one to 256 so that last sample will dropped for the inputs and first sample will be dropped from the outputs. This makes model to see only previous samples to predict next sample.

        For example:
                 input  :     295    4354   63    72      6035   63
                 output :     4354   63     72    6035    63     3170




In [8]:
context_size = 256
tokenizer.enable_padding(length=context_size+1, pad_id = 0, pad_token = "PAD")
tokenizer.enable_truncation(max_length=context_size+1)
#tokenizer.post_process()
tokenizer.decoder = decoders.BPEDecoder()
encoding = tokenizer.encode(raw_dataset["train"][50]["whole_func_string"])
print(encoding.ids)
print(encoding.attention_mask)

[296, 1590, 1814, 9, 249, 13, 2541, 275, 223, 291, 223, 311, 1843, 27, 232, 2862, 961, 462, 624, 1549, 257, 9746, 771, 2541, 223, 291, 223, 2186, 234, 1602, 15, 2471, 9, 564, 9, 2431, 492, 1183, 10, 223, 297, 227, 13, 1590, 254, 2022, 9, 2431, 275, 242, 1590, 325, 13, 1590, 350, 234, 1590, 15, 925, 15426, 242, 2186, 60, 85, 62, 234, 273, 15, 1407, 350, 9, 1407, 325, 10, 452, 1590, 350, 223, 303, 2186, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 



*   Generator to prepare inputs and outputs in the batches.
*   Inputs and Outputs will have sequences ids encoded from the tokenizer.



In [9]:
batch_size = 100
def generate_train_data():
  decoder_inputs = []
  decoder_targets = []
  dataset = raw_dataset["train"]
  for start_idx in range(0, len(dataset)-(len(dataset)%batch_size), batch_size):
      samples = dataset[start_idx : start_idx + batch_size]
      seqs = tokenizer.encode_batch(samples["whole_func_string"])
      decoder_inputs = [seq.ids[:-1] for seq in seqs] # Drop the last token in the sentence.
      decoder_targets = [seq.ids[1:] for seq in seqs]  # Drop the first token in the sentence.
      yield decoder_inputs, decoder_targets


In [ ]:
#tf_dataset = raw_dataset["train"].to_tf_dataset(batch_size = 100, columns = ['whole_func_string'])
#tf_dataset

<_PrefetchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [10]:
iterator = iter(generate_train_data())
decoder_inputs, decoder_targets = next(iterator)
print(decoder_inputs[80])


[296, 628, 53, 19, 49, 13070, 9, 1318, 275, 233, 291, 233, 10198, 232, 19537, 239, 7288, 312, 232, 869, 7336, 3996, 15, 313, 311, 395, 11349, 27, 311, 693, 2311, 4536, 408, 5596, 65, 233, 311, 781, 27, 311, 693, 2311, 47, 396, 239, 49, 13070, 65, 313, 20347, 302, 498, 23111, 8517, 3098, 233, 573, 6314, 15, 78, 660, 1652, 1194, 3884, 15, 2937, 74, 15, 241, 300, 73, 321, 15, 11017, 16, 23702, 16, 339, 620, 12013, 16, 15679, 2562, 16, 15679, 2562, 15, 1826, 7565, 233, 4548, 586, 8899, 90, 628, 890, 66, 337, 351, 691, 280, 348, 4513, 12013, 15, 233, 291, 233, 309, 69, 13, 487, 10, 234, 636, 15, 5909, 15, 12658, 9, 1318, 15, 1318, 10, 233, 524, 234, 636, 15, 784, 930, 69, 60, 18, 548, 271, 60, 17, 548, 271, 60, 19, 4212, 233, 642, 234, 636, 15, 784, 5774, 87, 60, 18, 13, 397, 548, 416, 87, 60, 18, 13, 444, 548, 416, 87, 60, 18, 13, 555, 4937, 1798, 405, 87, 60, 19, 13, 397, 548, 416, 87, 60, 19, 13, 444, 548, 416, 87, 60, 19, 13, 555, 4937, 1798, 4867, 87, 60, 17, 13, 397, 548, 487, 60, 17,

In [11]:
#tf_dataset = tf.data.Dataset.from_tensor_slices((decoder_inputs, decoder_targets)).batch(10, drop_remainder=True)

tf_dataset = tf.data.Dataset.from_generator(generate_train_data, output_types=(tf.int32, tf.int32), output_shapes=(tf.TensorShape([batch_size, context_size]), tf.TensorShape([batch_size, context_size])))

# checks to see if data is loading properly.
iterator = iter(tf_dataset)
ins, outs = next(iterator)
print(ins.shape)
next(iterator)[0]

(100, 256)


<tf.Tensor: shape=(100, 256), dtype=int32, numpy=
array([[ 296,  410, 4181, ...,    0,    0,    0],
       [ 296, 2704,   64, ...,    0,    0,    0],
       [ 296, 5615,   64, ...,    0,    0,    0],
       ...,
       [ 296, 2722,   64, ...,    0,    0,    0],
       [ 296,  550,    9, ...,    0,    0,    0],
       [ 296, 4521,    9, ...,    0,    0,    0]], dtype=int32)>

Multi Head Attention



*   Each Attention head performs Scaled Dot Product Self-Attention operation where given Keys, Query and Values, the return matrix of values given by below operation.

        Attention(Q,K,V) = softmax((Q*Transpose(K))/sqrt(d))*V





In [12]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dims = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(key_dims)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores)
  return tf.matmul(weights, value), weights



**Generating queries, keys, and values for multiple heads.**

> Now that we have a way to calculate self-attention, let's actually generate the input queries, keys, and values for multiple heads.

>  each attention head had its own separate set of query, key, and value weights. Each weight matrix was of dimension  d x d/h  where h was the number of heads.




In [13]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.w_queries = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.w_keys = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.w_values = tf.keras.layers.Dense(self.d_model, use_bias=False)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])

  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, query, key, value, mask):
    queries = self.w_queries(query)
    keys = self.w_keys(key)
    values = self.w_values(value)

    queries = self.split_heads(queries)
    keys = self.split_heads(keys)
    values = self.split_heads(values)

    output, attn_weights = scaled_dot_product_attention(queries, keys, values, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


Feed Forward Neural Network

In [14]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

Decode Block

In [15]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()

  def call(self, input, training, decoder_mask):
    mhsa_output1, attn_weights = self.mhsa1(input, input, input, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + input)

    ffn_output = self.ffn(mhsa_output1)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(ffn_output + mhsa_output1)

    return output, attn_weights


Decoder with Mulitple Layers

In [16]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, input, training, decoder_mask):
    token_embeds = self.token_embed(input)

    seq_len = input.shape[1]
    # Generate position indices.
    num_pos = input.shape[0] * seq_len
    pos_idx = np.resize(np.arange(seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, input.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(x, training, decoder_mask)

    return x, weights

Custom loss function to remove effect of padding

In [17]:
def loss_func(targets, logits):
  ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)
  return ce_loss(targets, logits, sample_weight=mask)

In [18]:
class GPTModel(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_input_len, dropout_rate=0.1):
    super(GPTModel, self).__init__()

    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_input_len, dropout_rate)

    # The final dense layer to generate logits from the model output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  @tf.function
  def train_step(self, inputs):
      loss = 0.

      input_seq, targets = inputs
      with tf.GradientTape() as tape:

        dec_padding_mask = tf.cast(tf.math.not_equal(input_seq, 0), tf.float32)
        dec_padding_mask = dec_padding_mask[:, tf.newaxis, tf.newaxis, :]
        input_seq_len = len(input_seq[0])
        look_ahead_mask = tf.linalg.band_part(tf.ones((input_seq_len,
                                               input_seq_len)), -1, 0)
        dec_mask = tf.minimum(dec_padding_mask, look_ahead_mask)

        logits, _ = self.decoder(input_seq, True, dec_mask)
        logits =   self.output_layer(logits)
        loss += self.loss(targets, logits)

      # Update the parameters and the optimizer
      variables = self.decoder.trainable_variables
      gradients = tape.gradient(loss, variables)
      self.optimizer.apply_gradients(zip(gradients, variables))

      return {'loss': loss}

  def call(self, input, training):
    logits, _ = self.decoder(input, False, None)
    logits =   self.output_layer(logits)
    return logits



In [19]:
model = GPTModel(
    num_blocks = 6,
    d_model = 512,
    num_heads = 4,
    hidden_dim = 1024,
    target_vocab_size = tokenizer.get_vocab_size(),
    max_input_len = 256)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_func, run_eagerly=True)

In [ ]:
epochs = 1
model.fit(tf_dataset, epochs=epochs)

   2784/Unknown - 3891s 1s/step - loss: 5.9230

In [21]:
def top_k_logits(logits, k):
	if k == 0:
		return logits

	values, _ = tf.nn.top_k(logits, k=k)
	min_values = values[:, -1]

	return tf.where(logits < min_values, tf.ones_like(logits, dtype=logits.dtype) * -1e10, logits)

# Nucleas Sampling (https://arxiv.org/pdf/1904.09751.pdf)


def top_p_logits(logits, p):
	"""Took from OpenAI GPT-2 Implememtation"""
	batch = tf.shape(logits)[0]
	sorted_logits = tf.sort(logits, direction='DESCENDING', axis=-1)
	cumulative_probs = tf.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

	indices = tf.stack([
		tf.range(0, batch),
		tf.maximum(tf.reduce_sum(tf.cast(cumulative_probs <= p, tf.int32), axis=-1) - 1, 0),
	], axis=-1)

	min_values = tf.gather_nd(sorted_logits, indices)
	return tf.where(
		logits < min_values,
		tf.ones_like(logits) * -1e10,
		logits,
	)

In [23]:
def sample_sequence(context=None, seq_len=256, temperature=1,top_k=8,top_p=8, nucleus_sampling=False):
  if context == None:
    print("Give some context to model.................")
    return
  prev_ctx = tf.expand_dims(tokenizer.encode(context), 0)
  for i in range(seq_len):
    logits, past = model(prev_ctx, training=False)
    print(logits)
    logits = logits[:, -1, :] / tf.cast(temperature, tf.float32)
    print(logits)
    logits = top_k_logits(logits, k=top_k)
    print(logits)
    if nucleus_sampling:
      logits = top_p_logits(logits, p=top_p)

    samples = tf.random.categorical(logits, num_samples=1, dtype=tf.int32)
    # print(samples)

    # print("shape.........")
    # print(tf.shape(output))
    # print(tf.shape(samples))
    output = tf.concat([prev_ctx, samples], axis=-1)
    prev_ctx = output
    # print(tf.shape(output))
    # print(output)

  # print("--------------------------")
  result = tf.squeeze(output, axis=0)
  pred = [int(i) for i in result]
  generated_seq = tokenizer.decode(pred[1:])
  return generated_seq


In [24]:
seq = sample_sequence("def get_mean_and_stddevs(self, sites, rup, dists, imt, stddev_types):\n   ")
print(seq.ids)

ValueError: ignored